# Homework2

### Mandatory Task:
You are to solve the first sub-problem: to implement the A-Priori algorithm for finding frequent itemsets with support at least s in a dataset of sales transactions. Remind that support of an itemset is the number of transactions containing the itemset. To test and evaluate your implementation, write a program that uses your A-Priori algorithm implementation to discover frequent itemsets with support at least s in a given dataset of sales transactions.

In [5]:
!pip install findspark

  Using cached findspark-2.0.1-py2.py3-none-any.whl.metadata (352 bytes)
Using cached findspark-2.0.1-py2.py3-none-any.whl (4.4 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
from pyspark import SparkContext, SparkConf
import findspark

In [2]:
# Parameters:
s = 0.2 # support threshold

In [7]:
# initializing Spark
findspark.init()
conf = SparkConf().setAppName("FreqItemSets").setMaster("local[*]")
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/11/12 12:17:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
transactions = sc.textFile("datasets/transaction_dataset.txt").map(lambda line: line.strip().split(" "))
print(transactions.first())

In [20]:
def count_support(itemset, transactions):
    # Count occurrences of itemset in transactions
    return transactions.filter(lambda transaction: set(itemset).issubset(set(transaction))).count()


def apriori(transactions, s):
    trans_num = transactions.count()
    support_count = s * trans_num
    frequent_itemsets = []
    
    transactions = transactions.cache()
    single_itemsets = transactions.flatMap(lambda transaction: transaction).distinct()
    current_itemsets = single_itemsets.map(lambda item: [item])
    
    broadcast_transactions = sc.broadcast(transactions.collect())
    
    k=1 #size of the itemset
    while current_itemsets.count() > 0:
        itemset_support = current_itemsets.map(lambda itemset: (itemset, count_support(itemset, broadcast_transactions)))
        
        current_freq_itemset = itemset_support.filter(lambda itemset: itemset[1] >= support_count).map(lambda itemset: itemset[0]).collect()
        frequent_itemsets.extend(current_freq_itemset)
        
        k += 1
        candidate_itemsets = [
            sorted(set(i1).union(set(i2))) 
            for i1 in current_freq_itemset for i2 in current_freq_itemset
            if len(set(i1).union(set(i2))) == k
        ]
        
        # Use sc.parallelize to create an RDD from the candidate itemsets
        current_itemsets = sc.parallelize(candidate_itemsets).distinct()

    return frequent_itemsets
    

In [21]:
frequent_itemsets = apriori(transactions, s)

24/11/12 12:51:31 ERROR Executor: Exception in task 1.0 in stage 13.0 (TID 24)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/kovacsanna/.local/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/Users/kovacsanna/.local/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Users/kovacsanna/.local/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/kovacsanna/.local/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 81, in wrapper
    return f(*args, **kwargs)
  File "/var/folders/79/s600fth94m50jws11s3p_ggm0000gn/T/ipykernel_16108/1020662148.py", line 19, in <lambda>
  File "/var/folders/79/s600fth94m50jws11s3p_ggm0000gn/T/

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 13.0 failed 1 times, most recent failure: Lost task 1.0 in stage 13.0 (TID 24) (macbooks-air executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/kovacsanna/.local/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/Users/kovacsanna/.local/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Users/kovacsanna/.local/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/kovacsanna/.local/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.p